### Training Code
This ipynb file trains the neural network. The model is described in the model() function, and also described below. Remember that training can take a lot of time, and you may not want to run it in interactive mode 
(Ipynotebook).

If you get errors when running this code, restart the kernel and clear the outputs, then run all cells. You may also need to delete the traning data in /data/tensorflow_sessions and re-train.

### Model  
| **Fully connected 1 (528 neurons)** |
| :---: |
| ReLu |
| **Fully connected 2 (786 neurons)** |
| ReLu |
| **Fully connected 3 (1248 neurons)**  |
| ReLu |
| Dropout |
| **Softmax_linear** |

### Explanation of the code in model.py
The model is loaded by calling the model(NUM_CLASSES)  
(NUM_CLASSES = 6 in this example: {"Relax", "Ok", "Fist", "Like", "Rock", "Spock"}  

The function returns the following variables:  
x - placeholder variable containing 64 columns (features) and variable rows (samples)  
y - placeholder variable containing NUM_CLASSES columns (labels) and variable rows (samples)  
output - output of neural network (column x is 1 if class is x)  
global_step - variable having a value of 0  
y_pred_class - argmax of output for sample to indicate assigned class

### Import modules and dataset

In [1]:
import numpy as np
import tensorflow as tf
from time import time
from include.data import get_data_set
from include.model import model

train_x, train_y = get_data_set()

In [2]:
# Vars
_NUM_ITERATIONS = 20000
_BATCH_SIZE = 300
_CLASS_SIZE = 6
_SAVE_PATH = "./data/tensorflow_sessions/myo_armband/"

In [3]:
# Load model
x, y, output, global_step, y_pred_cls = model(_CLASS_SIZE)

Instructions for updating:
Use the `axis` argument instead


In [4]:
# Setup TF Session
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))
tf.summary.scalar("Loss", loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-4).minimize(loss, global_step=global_step)

correct_prediction = tf.equal(y_pred_cls, tf.argmax(y, dimension=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar("Accuracy/train", accuracy)

saver = tf.train.Saver()
init = tf.global_variables_initializer()
merged = tf.summary.merge_all()
sess = tf.Session()
train_writer = tf.summary.FileWriter(_SAVE_PATH, sess.graph)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [5]:
# OPTIONAL Check if checkpoint exists so we continue from last session
Retrain = True
if(Retrain):
    sess.run(tf.global_variables_initializer())
    print("Variables Initialized.")
else:
    try:
        print("Trying to restore last checkpoint ...")
        last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=_SAVE_PATH)
        saver.restore(sess, save_path=last_chk_path)
        print("Restored checkpoint from:", last_chk_path)
    except:
        sess.run(tf.global_variables_initializer())
        print("Failed to restore checkpoint. Variables initialized.")

Variables Initialized.


### Define function that trains

This function picks _BATCH_SIZE samples randomly from the dataset, and trains to that batch, repeating this process _NUM_ITERATIONS times. If num_iterations is not specified when calling, it iterates a 1000 times.

In [6]:
tf.reset_default_graph()

In [7]:
def train(num_iterations = 1000):
    for i in range(num_iterations):
        randidx = np.random.randint(len(train_x), size=_BATCH_SIZE)
        batch_xs = train_x[randidx]
        batch_ys = train_y[randidx]

        start_time = time()
        i_global, _ = sess.run([global_step, optimizer], feed_dict={x: batch_xs, y: batch_ys})
        duration = time() - start_time

        if (i_global % 10 == 0) or (i == num_iterations - 1):
            _loss, batch_acc = sess.run([loss, accuracy], feed_dict={x: batch_xs, y: batch_ys})
            msg = "Global Step: {0:>6}, accuracy: {1:>6.1%}, loss = {2:.2f} ({3:.1f} examples/sec, {4:.2f} sec/batch)"
            print(msg.format(i_global, batch_acc, _loss, _BATCH_SIZE / duration, duration))

        if (i_global % 100 == 0) or (i == num_iterations - 1):
            data_merged, global_1 = sess.run([merged, global_step], feed_dict={x: batch_xs, y: batch_ys})
            train_writer.add_summary(data_merged, global_1)
            saver.save(sess, save_path=_SAVE_PATH, global_step=global_step)
            print("Saved checkpoint.")

### Train the data
## Warning: This cell will take a long time to complete. (~20 mins / 1k iteration for GT680m)

In [8]:
%%time
train(_NUM_ITERATIONS)
print('Training Complete')

Global Step:     10, accuracy:  22.7%, loss = 1397208.75 (6245.8 examples/sec, 0.05 sec/batch)
Global Step:     20, accuracy:  17.3%, loss = 1400288.38 (5878.4 examples/sec, 0.05 sec/batch)
Global Step:     30, accuracy:  15.3%, loss = 1314154.50 (4542.4 examples/sec, 0.07 sec/batch)
Global Step:     40, accuracy:  23.0%, loss = 1170581.75 (6117.6 examples/sec, 0.05 sec/batch)
Global Step:     50, accuracy:  18.3%, loss = 1394577.12 (4164.3 examples/sec, 0.07 sec/batch)
Global Step:     60, accuracy:  20.7%, loss = 1267743.25 (6517.6 examples/sec, 0.05 sec/batch)
Global Step:     70, accuracy:  18.7%, loss = 1193608.12 (5259.2 examples/sec, 0.06 sec/batch)
Global Step:     80, accuracy:  16.0%, loss = 1268704.88 (4542.1 examples/sec, 0.07 sec/batch)
Global Step:     90, accuracy:  19.7%, loss = 1186658.12 (4684.4 examples/sec, 0.06 sec/batch)
Global Step:    100, accuracy:  18.7%, loss = 1274864.00 (6118.5 examples/sec, 0.05 sec/batch)
Saved checkpoint.
Global Step:    110, accuracy:  

Global Step:    870, accuracy:  31.3%, loss = 528107.00 (5552.5 examples/sec, 0.05 sec/batch)
Global Step:    880, accuracy:  31.0%, loss = 533504.94 (4684.6 examples/sec, 0.06 sec/batch)
Global Step:    890, accuracy:  31.3%, loss = 577266.25 (4996.5 examples/sec, 0.06 sec/batch)
Global Step:    900, accuracy:  33.7%, loss = 539638.00 (4612.3 examples/sec, 0.07 sec/batch)
Saved checkpoint.
Global Step:    910, accuracy:  36.7%, loss = 491361.72 (4612.2 examples/sec, 0.07 sec/batch)
Global Step:    920, accuracy:  35.7%, loss = 469637.59 (5168.9 examples/sec, 0.06 sec/batch)
Global Step:    930, accuracy:  32.7%, loss = 549611.94 (5081.2 examples/sec, 0.06 sec/batch)
Global Step:    940, accuracy:  35.3%, loss = 529712.00 (5353.1 examples/sec, 0.06 sec/batch)
Global Step:    950, accuracy:  39.3%, loss = 516039.47 (4344.7 examples/sec, 0.07 sec/batch)
Global Step:    960, accuracy:  38.0%, loss = 534304.31 (5259.1 examples/sec, 0.06 sec/batch)
Global Step:    970, accuracy:  37.0%, los

Global Step:   1730, accuracy:  45.3%, loss = 320063.47 (5352.8 examples/sec, 0.06 sec/batch)
Global Step:   1740, accuracy:  34.3%, loss = 319953.94 (5259.4 examples/sec, 0.06 sec/batch)
Global Step:   1750, accuracy:  44.7%, loss = 304960.00 (5353.4 examples/sec, 0.06 sec/batch)
Global Step:   1760, accuracy:  44.3%, loss = 311298.38 (4996.7 examples/sec, 0.06 sec/batch)
Global Step:   1770, accuracy:  45.3%, loss = 359663.00 (4996.7 examples/sec, 0.06 sec/batch)
Global Step:   1780, accuracy:  41.7%, loss = 324850.28 (4408.9 examples/sec, 0.07 sec/batch)
Global Step:   1790, accuracy:  45.0%, loss = 324513.06 (5353.6 examples/sec, 0.06 sec/batch)
Global Step:   1800, accuracy:  45.3%, loss = 293360.69 (5450.3 examples/sec, 0.06 sec/batch)
Saved checkpoint.
Global Step:   1810, accuracy:  44.0%, loss = 305855.06 (3612.0 examples/sec, 0.08 sec/batch)
Global Step:   1820, accuracy:  44.7%, loss = 327457.53 (4542.7 examples/sec, 0.07 sec/batch)
Global Step:   1830, accuracy:  45.0%, los

Global Step:   2590, accuracy:  50.0%, loss = 237110.45 (4051.2 examples/sec, 0.07 sec/batch)
Global Step:   2600, accuracy:  52.3%, loss = 192672.42 (5081.3 examples/sec, 0.06 sec/batch)
Saved checkpoint.
Global Step:   2610, accuracy:  48.3%, loss = 249144.61 (3294.6 examples/sec, 0.09 sec/batch)
Global Step:   2620, accuracy:  48.3%, loss = 192241.33 (5081.4 examples/sec, 0.06 sec/batch)
Global Step:   2630, accuracy:  56.3%, loss = 177929.86 (5169.0 examples/sec, 0.06 sec/batch)
Global Step:   2640, accuracy:  51.7%, loss = 166678.70 (5450.9 examples/sec, 0.06 sec/batch)
Global Step:   2650, accuracy:  54.0%, loss = 217392.38 (4474.6 examples/sec, 0.07 sec/batch)
Global Step:   2660, accuracy:  50.7%, loss = 222049.23 (5878.8 examples/sec, 0.05 sec/batch)
Global Step:   2670, accuracy:  50.7%, loss = 236994.30 (5877.5 examples/sec, 0.05 sec/batch)
Global Step:   2680, accuracy:  52.7%, loss = 212194.31 (5878.0 examples/sec, 0.05 sec/batch)
Global Step:   2690, accuracy:  49.7%, los

Global Step:   3450, accuracy:  57.7%, loss = 126459.32 (5081.5 examples/sec, 0.06 sec/batch)
Global Step:   3460, accuracy:  49.0%, loss = 175485.92 (5450.7 examples/sec, 0.06 sec/batch)
Global Step:   3470, accuracy:  54.7%, loss = 152249.12 (5877.1 examples/sec, 0.05 sec/batch)
Global Step:   3480, accuracy:  51.3%, loss = 158571.42 (4996.7 examples/sec, 0.06 sec/batch)
Global Step:   3490, accuracy:  52.3%, loss = 152529.97 (5552.5 examples/sec, 0.05 sec/batch)
Global Step:   3500, accuracy:  58.3%, loss = 131943.78 (5168.8 examples/sec, 0.06 sec/batch)
Saved checkpoint.
Global Step:   3510, accuracy:  56.0%, loss = 129978.28 (5169.0 examples/sec, 0.06 sec/batch)
Global Step:   3520, accuracy:  57.7%, loss = 117861.07 (4684.5 examples/sec, 0.06 sec/batch)
Global Step:   3530, accuracy:  55.7%, loss = 149271.61 (4758.8 examples/sec, 0.06 sec/batch)
Global Step:   3540, accuracy:  60.3%, loss = 147082.56 (4996.5 examples/sec, 0.06 sec/batch)
Global Step:   3550, accuracy:  52.7%, los

...

Global Step:  16490, accuracy:  83.7%, loss = 8309.11 (5878.5 examples/sec, 0.05 sec/batch)
Global Step:  16500, accuracy:  79.7%, loss = 11093.76 (5936.6 examples/sec, 0.05 sec/batch)
Saved checkpoint.
Global Step:  16510, accuracy:  86.0%, loss = 7801.76 (3701.2 examples/sec, 0.08 sec/batch)
Global Step:  16520, accuracy:  84.3%, loss = 8136.45 (4684.4 examples/sec, 0.06 sec/batch)
Global Step:  16530, accuracy:  83.0%, loss = 8018.29 (5936.6 examples/sec, 0.05 sec/batch)
Global Step:  16540, accuracy:  82.3%, loss = 8699.00 (5996.0 examples/sec, 0.05 sec/batch)
Global Step:  16550, accuracy:  84.3%, loss = 7431.39 (5996.0 examples/sec, 0.05 sec/batch)
Global Step:  16560, accuracy:  84.0%, loss = 8172.10 (5450.9 examples/sec, 0.06 sec/batch)
Global Step:  16570, accuracy:  83.3%, loss = 9096.63 (5878.4 examples/sec, 0.05 sec/batch)
Global Step:  16580, accuracy:  81.7%, loss = 8173.67 (5821.4 examples/sec, 0.05 sec/batch)
Global Step:  16590, accuracy:  83.3%, loss = 11300.50 (5936.

Global Step:  17370, accuracy:  82.3%, loss = 5986.86 (6118.3 examples/sec, 0.05 sec/batch)
Global Step:  17380, accuracy:  85.7%, loss = 8001.23 (6118.4 examples/sec, 0.05 sec/batch)
Global Step:  17390, accuracy:  84.7%, loss = 8074.87 (5878.4 examples/sec, 0.05 sec/batch)
Global Step:  17400, accuracy:  87.7%, loss = 6421.40 (5936.7 examples/sec, 0.05 sec/batch)
Saved checkpoint.
Global Step:  17410, accuracy:  83.7%, loss = 8862.06 (5306.2 examples/sec, 0.06 sec/batch)
Global Step:  17420, accuracy:  84.3%, loss = 6573.60 (5451.0 examples/sec, 0.06 sec/batch)
Global Step:  17430, accuracy:  82.0%, loss = 8640.72 (5603.8 examples/sec, 0.05 sec/batch)
Global Step:  17440, accuracy:  84.3%, loss = 8569.82 (5936.7 examples/sec, 0.05 sec/batch)
Global Step:  17450, accuracy:  82.3%, loss = 7202.66 (5936.6 examples/sec, 0.05 sec/batch)
Global Step:  17460, accuracy:  86.7%, loss = 6102.41 (5996.1 examples/sec, 0.05 sec/batch)
Global Step:  17470, accuracy:  87.3%, loss = 5734.20 (5656.7 

Global Step:  18250, accuracy:  84.0%, loss = 8194.45 (6181.4 examples/sec, 0.05 sec/batch)
Global Step:  18260, accuracy:  83.7%, loss = 9998.04 (5878.5 examples/sec, 0.05 sec/batch)
Global Step:  18270, accuracy:  84.3%, loss = 9622.51 (5821.3 examples/sec, 0.05 sec/batch)
Global Step:  18280, accuracy:  83.7%, loss = 6363.57 (5821.4 examples/sec, 0.05 sec/batch)
Global Step:  18290, accuracy:  86.3%, loss = 7334.53 (5765.4 examples/sec, 0.05 sec/batch)
Global Step:  18300, accuracy:  84.3%, loss = 8982.92 (5551.8 examples/sec, 0.05 sec/batch)
Saved checkpoint.
Global Step:  18310, accuracy:  87.3%, loss = 6790.66 (4135.2 examples/sec, 0.07 sec/batch)
Global Step:  18320, accuracy:  85.7%, loss = 4989.56 (5259.4 examples/sec, 0.06 sec/batch)
Global Step:  18330, accuracy:  83.3%, loss = 7402.26 (5995.9 examples/sec, 0.05 sec/batch)
Global Step:  18340, accuracy:  82.0%, loss = 8848.35 (5936.3 examples/sec, 0.05 sec/batch)
Global Step:  18350, accuracy:  86.3%, loss = 6810.68 (5765.3 

Global Step:  19130, accuracy:  88.0%, loss = 6384.69 (5996.0 examples/sec, 0.05 sec/batch)
Global Step:  19140, accuracy:  87.3%, loss = 6213.15 (4914.7 examples/sec, 0.06 sec/batch)
Global Step:  19150, accuracy:  88.0%, loss = 4555.38 (5996.0 examples/sec, 0.05 sec/batch)
Global Step:  19160, accuracy:  87.0%, loss = 6392.14 (5936.6 examples/sec, 0.05 sec/batch)
Global Step:  19170, accuracy:  82.7%, loss = 6027.79 (5710.7 examples/sec, 0.05 sec/batch)
Global Step:  19180, accuracy:  85.7%, loss = 5612.38 (5656.5 examples/sec, 0.05 sec/batch)
Global Step:  19190, accuracy:  86.0%, loss = 7568.10 (5306.2 examples/sec, 0.06 sec/batch)
Global Step:  19200, accuracy:  85.7%, loss = 4760.82 (5710.4 examples/sec, 0.05 sec/batch)
Saved checkpoint.
Global Step:  19210, accuracy:  90.3%, loss = 4924.83 (4835.5 examples/sec, 0.06 sec/batch)
Global Step:  19220, accuracy:  89.3%, loss = 4919.83 (5038.8 examples/sec, 0.06 sec/batch)
Global Step:  19230, accuracy:  87.7%, loss = 5470.45 (6118.4 

Saved checkpoint.
Training Complete
Wall time: 23min 24s


In [9]:
# Close the tf session
sess.close()